<a href="https://colab.research.google.com/github/maryamelnahas/CSEN711-MS1-Weir-Algorithm/blob/main/BINF711_MS1_Weir_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import re
from collections import Counter
import io

In [4]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving production.tsv to production (1).tsv
User uploaded file "production (1).tsv" with length 11818456 bytes


In [6]:


def load_and_clean_data(file_name: str) -> list:

    print(f"\n--- Starting Data Processing for '{file_name}' ---")

    try:
        df = pd.read_csv(file_name, sep='\t')
    except Exception as e:
        print(f"Initial pandas read failed: {e}")
        # Robust handling for files with internal encoding/quoting issues
        print("Attempting robust load by cleaning potential embedded characters...")

        # Read the file content manually
        with open(file_name, 'r', encoding='utf-8', errors='ignore') as f:
            content = f.read()

        # Replace the problematic string '###COMMA###' seen in the data snippet
        # and any remaining quote characters that might interfere with TSV parsing.
        content_cleaned = content.replace('###COMMA###', '').replace('"', '').replace("'", "")

        # Use io.StringIO to read the cleaned string data into pandas
        df = pd.read_csv(io.StringIO(content_cleaned), sep='\t')

    print(f"Initial rows loaded: {len(df)}")

    # --- 2. Data Cleaning ---

    # Assuming the password column is the first one based on the TSV header:
    password_col_name = df.columns[0]
    print(f"Identified password column: '{password_col_name}'")

    # 2a. Handle Missing Values
    initial_rows = len(df)
    df.dropna(subset=[password_col_name], inplace=True)
    rows_after_drop = len(df)

    if initial_rows != rows_after_drop:
        print(f"Dropped {initial_rows - rows_after_drop} rows with missing passwords.")

    # 2b. Convert to String Type
    # Ensure all entries are treated as strings for uniform character analysis
    df[password_col_name] = df[password_col_name].astype(str)

    # 2c. Deduplication (Optional but recommended for PCFG training)
    # PCFG relies on counting patterns, so using all passwords is fine,
    # but cleaning up leading/trailing whitespace is crucial.
    df[password_col_name] = df[password_col_name].str.strip()

    # 2d. Final Extraction
    password_list = df[password_col_name].tolist()

    print(f"--- Data Loading and Cleaning Complete ---")
    print(f"Total cleaned passwords for training: {len(password_list)}")
    print(f"First 5 sample passwords: {password_list[:5]}")

    return password_list


Dataset = "production.tsv"

training_passwords = load_and_clean_data(Dataset)


--- Starting Data Processing for 'production.tsv' ---
Initial rows loaded: 300000
Identified password column: 'password'
--- Data Loading and Cleaning Complete ---
Total cleaned passwords for training: 300000
First 5 sample passwords: ['5Ka392f0c29', '97DMs07gBQ3qP9Hwf3YxJxY3yFvoh1W9b6HrmSNfOB9rjUIZJJX7g0mrv0Rlv51r184odr317jq0df3MBIJ4B07kMK7JCx4QG7', 'qD2bN8VmEB', 'rJu42Kx', 'YloGVMJoL']


In [7]:
def segment_password(password: str) -> str:
    if not password:
        return ""

    # Regular expression pattern to find contiguous blocks of L, D, or S
    # Group 1: Letters ([a-zA-Z]+)
    # Group 2: Digits ([0-9]+)
    # Group 3: Symbols ([^a-zA-Z0-9]+)
    pattern = re.compile(r'([a-zA-Z]+)|([0-9]+)|([^a-zA-Z0-9]+)')

    segments = []

    # Iterate through all matches found in the password
    for match in pattern.finditer(password):
        segment = match.group(0)
        length = len(segment)

        # Determine which group matched and assign the corresponding symbol
        if match.group(1):
            segments.append(f'L{length}')
        elif match.group(2):
            segments.append(f'D{length}')
        elif match.group(3):
            segments.append(f'S{length}')

    return "".join(segments)


def generate_patterns(password_list: list) -> list:

    print("\n--- Starting Pattern Generation (Segmentation) ---")

    # Use a list comprehension for fast and efficient processing of the large list
    patterns = [segment_password(p) for p in password_list]

    print("Pattern generation complete.")

    return patterns



# 1. Load and Clean the data
training_passwords = load_and_clean_data(Dataset)

# 2. Generate the structural patterns
training_patterns = generate_patterns(training_passwords)

# --- Verification of Patterns ---
print(f"\nTotal structural patterns generated: {len(training_patterns)}")

# Show the first 5 raw passwords and their generated patterns
print("Verification (Raw Password -> Pattern):")
for i in range(5):
    print(f"  Raw: '{training_passwords[i]}' -> Pattern: '{training_patterns[i]}'")




--- Starting Data Processing for 'production.tsv' ---
Initial rows loaded: 300000
Identified password column: 'password'
--- Data Loading and Cleaning Complete ---
Total cleaned passwords for training: 300000
First 5 sample passwords: ['5Ka392f0c29', '97DMs07gBQ3qP9Hwf3YxJxY3yFvoh1W9b6HrmSNfOB9rjUIZJJX7g0mrv0Rlv51r184odr317jq0df3MBIJ4B07kMK7JCx4QG7', 'qD2bN8VmEB', 'rJu42Kx', 'YloGVMJoL']

--- Starting Pattern Generation (Segmentation) ---
Pattern generation complete.

Total structural patterns generated: 300000
Verification (Raw Password -> Pattern):
  Raw: '5Ka392f0c29' -> Pattern: 'D1L2D3L1D1L1D2'
  Raw: '97DMs07gBQ3qP9Hwf3YxJxY3yFvoh1W9b6HrmSNfOB9rjUIZJJX7g0mrv0Rlv51r184odr317jq0df3MBIJ4B07kMK7JCx4QG7' -> Pattern: 'D2L3D2L3D1L2D1L3D1L5D1L5D1L1D1L1D1L8D1L8D1L1D1L3D1L3D2L1D3L3D3L2D1L2D1L4D1L1D2L3D1L3D1L2D1'
  Raw: 'qD2bN8VmEB' -> Pattern: 'L2D1L2D1L4'
  Raw: 'rJu42Kx' -> Pattern: 'L3D2L2'
  Raw: 'YloGVMJoL' -> Pattern: 'L9'


In [8]:
def train_pcfg_model(training_patterns: list) -> tuple:

    print("\n--- Starting PCFG Model Training (Phase 3) ---")

    # 1. Establish Total Count
    total_trained_count = len(training_patterns)
    if total_trained_count == 0:
        print("Error: No patterns to train on.")
        return {}, 0

    # 2. Count Frequencies
    # Counter is the most efficient way to perform the counting task.
    pattern_counts = Counter(training_patterns)

    # 3. Calculate Probabilities (Maximum Likelihood Estimate)
    pcfg_model = {
        pattern: count / total_trained_count
        for pattern, count in pattern_counts.items()
    }

    unique_patterns = len(pcfg_model)

    print(f"Total unique structural patterns learned: {unique_patterns}")

    # Verification: Show the most common (and therefore weakest) patterns
    print("\nTop 5 Most Frequent Patterns (The Weakest Structures):")

    # Sort the model by probability in descending order
    sorted_model = sorted(pcfg_model.items(), key=lambda item: item[1], reverse=True)

    for pattern, probability in sorted_model[:5]:
        # Convert probability to percentage for easier interpretation
        percent = probability * 100
        print(f"  Pattern: {pattern.ljust(15)} | Count: {pattern_counts[pattern]} | Prob: {probability:.6f} ({percent:.2f}%)")

    print("\n--- PCFG Model Training Complete ---")

    return pcfg_model, total_trained_count


training_passwords = load_and_clean_data(Dataset)
training_patterns = generate_patterns(training_passwords)
pcfg_model, total_trained_count = train_pcfg_model(training_patterns)

# The 'pcfg_model' dictionary is the final output of this phase.
# It contains the probability (P) for every structural pattern (G) found in your dataset.
# Example: pcfg_model['L5D2'] = 0.0015


--- Starting Data Processing for 'production.tsv' ---
Initial rows loaded: 300000
Identified password column: 'password'
--- Data Loading and Cleaning Complete ---
Total cleaned passwords for training: 300000
First 5 sample passwords: ['5Ka392f0c29', '97DMs07gBQ3qP9Hwf3YxJxY3yFvoh1W9b6HrmSNfOB9rjUIZJJX7g0mrv0Rlv51r184odr317jq0df3MBIJ4B07kMK7JCx4QG7', 'qD2bN8VmEB', 'rJu42Kx', 'YloGVMJoL']

--- Starting Pattern Generation (Segmentation) ---
Pattern generation complete.

--- Starting PCFG Model Training (Phase 3) ---
Total unique structural patterns learned: 171474

Top 5 Most Frequent Patterns (The Weakest Structures):
  Pattern: L6              | Count: 4663 | Prob: 0.015543 (1.55%)
  Pattern: L7              | Count: 3520 | Prob: 0.011733 (1.17%)
  Pattern: L8              | Count: 3172 | Prob: 0.010573 (1.06%)
  Pattern: L5              | Count: 3158 | Prob: 0.010527 (1.05%)
  Pattern: L9              | Count: 1638 | Prob: 0.005460 (0.55%)

--- PCFG Model Training Complete ---
